In [1]:
# Импорт стандартных библиотек
import logging
import os
import uuid
from pathlib import Path

# Импорт библиотеки для загрузки переменных окружения из файла .env
from dotenv import load_dotenv

# Импорт аннотаций типов
from typing import Any, Dict, List

# Импорт внешних библиотек
import openai
from pydantic import BaseModel

# Импорт библиотек LangChain
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.schema import AgentAction, AgentFinish
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains.combine_documents.map_reduce import MapReduceDocumentsChain
from langchain.docstore.document import Document

# Загрузка переменных окружения из файла .env
load_dotenv()

# настройка логгирования
logging.basicConfig(level=logging.INFO)

In [2]:
# Получение API ключей из переменной окружения
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')
LANGCHAIN_TRACING_V2     = os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_API_KEY        = os.getenv('LANGCHAIN_API_KEY')
TAVILY_API_KEY           = os.getenv('TAVILY_API_KEY')
OPENAI_API_KEY           = os.environ['OPENAI_API_KEY']

In [3]:
openai.api_key = OPENAI_API_KEY

In [4]:
class FileManager:
    """
    Description:
        Класс для управления файлами, включая чтение, запись и добавление содержимого.
    """
    logging.basicConfig(level=logging.INFO)

    def __init__(self, working_directory: str = 'temp'):
        """
        Description:
            Инициализация рабочей директории.

        Args:
            working_directory: Путь к рабочей директории.
        """
        # Создаем рабочую директорию
        self.working_directory = Path(working_directory).absolute()
        logging.info("WORKING_DIRECTORY: %s", self.working_directory)

    @staticmethod
    def generate_run_id() -> str:
        """
        Description:
            Генерирует уникальный UUID.

        Returns:
            str: Сгенерированный UUID.
        """
        return str(uuid.uuid4())

    def read_document(self, file_name: str) -> str:
        """
        Description:
            Читает и возвращает содержимое файла.

        Args:
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        return FileManager._read_document(self.working_directory, file_name)

    @staticmethod
    def _read_document(working_directory: Path, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для чтения содержимого файла.

        Args:
            working_directory: Рабочая директория.
            file_name (str): Имя файла для чтения.

        Returns:
            str: Содержимое файла.
        """
        # Создаем путь к файлу с учетом рабочей директории
        file_path = working_directory / file_name.lstrip('/')
        logging.info(f"Attempting to read file from path: {file_path}")

        try:
            # Открываем файл для чтения
            with file_path.open("r", encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"File {file_name} not found at path: {file_path}")
            return f"File {file_name} not found."

    def write_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Создает и сохраняет текстовый документ.

        Args:
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        return FileManager._write_document(self.working_directory, content, file_name)

    @staticmethod
    def _write_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для записи документа.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для записи в файл.
            file_name: Имя файла для сохранения.

        Returns:
            str: Сообщение о сохранении файла.
        
        Raises:
            FileNotFoundError: Если файл не найден.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для записи
            with file_path.open("w", encoding='utf-8') as file:
                file.write(content)
            return f"Document saved to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error writing to file {file_name}: {e}")
            return f"Error writing to file {file_name}: {e}"

    def append_document(self, content: str, file_name: str) -> str:
        """
        Description:
            Добавляет содержимое в конец существующего файла.

        Args:
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        return FileManager._append_document(self.working_directory, content, file_name)

    @staticmethod
    def _append_document(working_directory: Path, content: str, file_name: str) -> str:
        """
        Description:
            Вспомогательный метод для добавления содержимого в документ.

        Args:
            working_directory: Рабочая директория.
            content: Текстовое содержимое для добавления в файл.
            file_name: Имя файла для добавления содержимого.

        Returns:
            str: Сообщение о сохранении файла.
        """
        # Создаем путь к файлу и необходимые директории
        file_path = working_directory / file_name.lstrip('/')
        file_path.parent.mkdir(parents=True, exist_ok=True)
        
        try:
            # Открываем файл для добавления содержимого
            with file_path.open("a", encoding='utf-8') as file:
                file.write(content)
            return f"Document appended to {file_name}"
        except IOError as e:
            # Логируем ошибку и возвращаем сообщение об ошибке
            logging.error(f"Error appending to file {file_name}: {e}")
            return f"Error appending to file {file_name}: {e}"

In [40]:
class AgentState():
    """
    Класс для представления состояния агента.
    """
    def __init__(self):
        self.messages: List[Dict[str, Any]] = []

class BaseAgent(AgentState):
    """
    Базовый класс для создания агентов.
    """

    def __init__(self, llm, system_prompt, tools: List[Any] = None):
        """
        Инициализация агента.

        Args:
            tools: Список инструментов, доступных агенту.
        """
        super().__init__()
        self.system_prompt = system_prompt
        self.llm = llm
        self.tools = tools or []

    def process_message(self, message: Dict[str, Any]) -> Dict[str, Any]:
        """
        Обрабатывает входящее сообщение и возвращает ответ.

        Args:
            message: Входящее сообщение.

        Returns:
            Ответ агента.
        """
        # Добавляем входящее сообщение в историю
        self.messages.append({"role": "user", "content": message["content"]})
        
        response = self.llm.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", 
                 "content": self.system_prompt},
                *self.messages
            ]
        ).choices

        # Добавляем ответ агента в историю
        self.messages.append({"role": "assistant", "content": response})

        return {"content": response}

    def call_tool(self, tool_name: str, **kwargs) -> Any:
        """
        Вызывает инструмент по имени.

        Args:
            tool_name: Имя инструмента.
            **kwargs: Дополнительные аргументы для инструмента.

        Returns:
            Результат работы инструмента.
        """
        for tool in self.tools:
            if tool.__class__.__name__ == tool_name:
                return tool.run(**kwargs)
        raise ValueError(f"Инструмент '{tool_name}' не найден.")

In [52]:
def pdf_loader(file_path: str) -> List[str]:
    """
    Description:
      Загружает PDF-файл и возвращает список документов.

    Args:
        file_path: Путь к PDF-файлу.

    Returns:
        Список документов, загруженных из PDF-файла.

    Raises:
        FileNotFoundError: Если указанный файл не найден.
        ValueError: Если файл не является допустимым PDF.

    Examples:
        >>> pdf_loader("example.pdf")
        ['Document content as a string.']
    """
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    
    return docs


In [58]:
# Загрузка PDF-файла
pdf_path = '/Users/cyberrunner/Downloads/Automatic Prompt Optimization with “Gradient Descent” and Beam Search.pdf'
pdf_pages = pdf_loader(pdf_path)

# Используем абсолютный путь к текущей директории
file_manager = FileManager(working_directory=os.getcwd())

# Инициализация агента
agent = BaseAgent(
    llm=openai,
    system_prompt=file_manager.read_document('prompts/system_prompt.txt'),
    tools=[]
)

# Итеративная суммаризация
summary = ""
for page in pdf_pages:
    summarized_content = agent.process_message(file_manager.read_document('prompts/chank_prompt.txt') + "\n" + page.page_content)
    summary += summarized_content + "\n"
    file_manager.append_document(summarized_content, 'summary.md')

# Запись окончательной суммаризации
file_manager.write_document(summary, 'final_summary.md')

### **Чанк 1:**

## **Проблема написания подсказок для LLM**

В этом чанке обсуждается проблема написания подсказок для больших языковых моделей (LLM), которая требует значительных усилий и знаний. Сложность заключается в том, что пользователи должны вручную экспериментировать с разными формулировками подсказок, что делает процесс крайне трудоемким.

Основная концепция этого чанка заключается в том, что процесс написания подсказок для LLMs является сложным и требует автоматизации. Эта проблема затрагивает необходимость создания более эффективных инструментов для улучшения качества работы LLM, чтобы пользователи могли тратить меньше времени на ручной процесс тестирования и редактирования подсказок.

### **Ключевая концепция: Необходимость автоматизации написания подсказок для LLM**

Автоматизация процесса написания подсказок для больших языковых моделей (LLMs) может значительно улучшить эффективность работы с этими системами. На данный момент глубокое понимание языка и моделирования текстов требует от пользователей экспертизы и времени для изучения, как формулировать подсказки, чтобы получать необходимые результаты.

Примером проблемы может служить ситуация, когда пользователю необходимо задать запрос на получение информации о конкретной теме. Например, для получения полного ответа о переезде в другую страну, пользователю может потребоваться попробовать множество вариантов формулировок, прежде чем он найдет тот, который дает удовлетворительный ответ от модели.

Чтобы упростить этот процесс, разработаны автоматизированные подходы, позволяющие улучшать подсказки с использованием алгоритмов, оптимизации и анализа данных. Это позволяет пользователям не быть специалистами в области обработки естественного языка, а просто использовать модель с более точными и понятными запросами.

### **Методы оптимизации подсказок**

Одним из предложенных способов оптимизации подсказок является использование алгоритма "Оптимизация Подсказок с Текстовыми Градиентами" (ProTeGi). Этот алгоритм вдохновлен концепцией численного градиентного спуска, которая позволяет автоматически интегрировать данные о том, как улучшить текущую подсказку, используя методы обучения на примерах.

Далее, в случае, если в следующем чанке содержатся описания конкретного алгоритма, это будет важно для понимания того, как именно ProTeGi работает и как его применение может улучшить результаты работы LLM.
### **Чанк 2:**

**Предыдущий контекст:** В предыдущем чанке речь шла о предложенном алгоритме ProTeGi, который улучшает оптимизацию текстовых подсказок, используя подход, подобный градиентному спуску, путем обработки обратной связи от больших языковых моделей (LLM) и редактирования подсказок на основе этой информации.

## **Дискретная оптимизация подсказок с использованием "градиентного спуска"**

Ключевой концепцией этого чанка является метод "дискретной оптимизации подсказок", который использует концепцию градиентного спуска для итеративного улучшения текстовых подсказок. Это позволяет создавать более точные и эффективные подсказки для работы с большим объемом данных, как например текстовые пары.

**Объяснение концепции:**

Градиентный спуск — это метод оптимизации, широко применяемый в машинном обучении для нахождения локального минимума функции потерь. В контексте ProTeGi процесс градиентного спуска адаптирован для работы с текстовыми подсказками. Он включает три ключевых шага:

1. **Оценка подсказки:** Алгоритм начинает с начальной подсказки $p_0$ и оценивает её, используя набор данных (или мини-батч).
2. **Создание локального "сигнала потерь":** На основе результатов вычисляется сигнал потерь, который служит для обозначения того, как текущая подсказка $p_0$ может быть улучшена. Это фактически "градиент", то есть описание ошибок подсказки по сравнению с желаемыми результатами.
3. **Редактирование подсказки:** Затем подсказка редактируется в противоположном семантическом направлении к градиенту, чтобы улучшить качество текстового результата в следующей итерации.

Таким образом, процесс градиентного спуска здесь фокусируется на цикла итеративного улучшения текстового контента. 

**Математическая формализация:**

В этом случае, формально описывается процесс как:
- \( p^* = \arg\max_{p \in \mathcal{L}} m(p, D_{te}) \)

Где:
- \( p^* \) — это оптимальная подсказка, которую мы хотим найти.
- \( m(p, D_{te}) \) — это функция, которая измеряет качество подсказки \( p \) на тестовых данных \( D_{te} \).
- \( \mathcal{L} \) — пространство допустимых подсказок.

**Пример и объяснение кода:**

Пусть у нас есть функция, которая имитирует процесс градиентного спуска для текстовых подсказок. Вот простой псевдокод:

```python
def optimize_prompt(initial_prompt, data_batch):
    prompt = initial_prompt  # Начальная подсказка
    for iteration in range(max_iterations):
        # Шаг 1: Оценка подсказки
        predictions = model(prompt, data_batch)  # Получаем предсказания

        # Шаг 2: Создание сигнала потерь (градиента)
        loss_signal = compute_loss_signal(predictions, data_batch)  # Функция вычисляет сигнал потерь

        # Шаг 3: Редактирование подсказки в противоположном направлении
        prompt = edit_prompt(prompt, loss_signal)  # Редактируем подсказку

    return prompt  # Возвращаем оптимизированную подсказку

def compute_loss_signal(predictions, data_batch):
    # Анализируем ошибки и создаем текстовое описание улучшений
    # (это быстрая заглушка, реальные детали будут зависеть от задачи и модели)
    return [generate_feedback(pred) for pred in predictions]

def edit_prompt(prompt, loss_signal):
    # Редактируем подсказку на основе сигнала потерь
    # (это также заглушка, реальная логика редактирования может быть сложной)
    return prompt + " " + improve_based_on_signal(loss_signal)
```

- `optimize_prompt`: Функция, которая инициализирует подсказку и выполняет итеративный процесс оптимизации.
- `compute_loss_signal`: Эта функция вычисляет сигнал потерь на основе предсказаний модели и набора данных.
- `edit_prompt`: Исходя из сигнала потерь, мы создаем новые редакции подсказки.

Таким образом, здесь мы видим, как градиентный спуск адаптирован для работы с текстами и как код иллюстрирует предложенный алгоритм.
### **Чанк 2:**

**Предыдущий контекст:** В предыдущем чанке обсуждался процесс создания градиентов в текстовом формате, обозначаемых как $g$, которые служат для указания направлений в семантическом пространстве, ухудшающих текущую подсказку $p_0$. Эти градиенты используются для редактирования подсказки с целью устранения обнаруженных проблем.

## **Поиск Промтов с Помощью Текстовых Градиентов (ProTeGi)**

В данном чанке описывается использование метода "лучевого поиска" (beam search) для оптимизации задач, связанных с подсказками в контексте обучения с помощью текстовых градиентов. Процесс состоит из нескольких итераций, где в каждой итерации текущая подсказка $p_0$ используется для генерации новых кандидатных подсказок, которые затем отбираются для дальнейшей работы.

### Математическая формализация

Алгоритм оптимизации подсказок представлен в виде алгоритма 1 (Algorithm 1). Он использует некоторые параметры:

- $p_0$: начальная подсказка.
- $b$: ширина луча, то есть максимальное количество подсказок, которые будут изучены на каждом шаге.
- $r$: глубина поиска, количество итераций.
- $m$: функция оценки, которая помогает выбрать наиболее подходящие подсказки.

Основные операции формализованы следующим образом:

1. Начинаем с установки луча $B_0 = \{p_0\}$.
2. Для каждой итерации $i$ от 1 до $r-1$:
   - Генерируем новые кандидатные подсказки через операцию Expand.
   - Выбираем $b$ наилучших кандидатов для следующей итерации на основе функции оценки $m$.
   
Финальная подсказка определяется как $\hat{p} = \arg\max_{p \in B_r} m(s)$.

### Пример кода

Для более глубокого понимания, приведем простой пример кода, реализующего алгоритм поиска подсказок:

```python
def beam_search(initial_prompt, beam_width, search_depth, metric_function):
    # Шаг 1: Initialize the beam with the initial prompt
    B = [initial_prompt]

    # Шаг 2: Start the iterative process
    for i in range(search_depth):
        C = []  # Список для хранения новых кандидатных подсказок
        for p in B:
            # Генерация новых кандидатов на основе текущей подсказки
            new_candidates = expand_prompt(p)  # Функция генерации новых подсказок
            C.extend(new_candidates)  # Добавление новых кандидатов в общий список
            
        # Шаг 3: Выбор b лучших подсказок по заданной метрике
        B = select_best_candidates(C, beam_width, metric_function)

    # Шаг 4: Назначаем финальную подсказку
    best_prompt = max(B, key=metric_function)  # Выбор наилучшей подсказки
    return best_prompt

def expand_prompt(prompt):
    # Функция, генерирующая новые кандидатные подсказки
    # В данной функции можно добавить логику генерации на основе ошибок и градиентов
    return [prompt + " variation 1", prompt + " variation 2"]

def select_best_candidates(candidates, beam_width, metric_function):
    # Сортирует кандидатов по метрике и выбирает наилучшие
    return sorted(candidates, key=metric_function)[:beam_width]
```

**Комментарии к коду:**
- `beam_search` — основная функция, реализующая лучевой поиск.
- `initial_prompt` — исходная подсказка.
- `beam_width` — заданная ширина луча.
- `search_depth` — количество итераций.
- `metric_function` — функция, позволяющая оценить качество подсказок.
- `expand_prompt` — пример функции, генерирующей вариации текущей подсказки.
- `select_best_candidates` — функция, выбирающая лучшие подсказки на основе их оценок.

Таким образом, описанный процесс позволяет систематически улучшать подсказки на основе их оценок в многокомпонентном процессе оптимизации.
### **Чанк 2:**

**Предыдущий контекст:** В предыдущем чанке обсуждалась аналогия между задачей минимизации количества запросов к модели и известной проблемой определения "лучшего оружия" в области оптимизации многоруких бандитов. Затрагивались методы, используемые для оценки производительности предложений на различных подмножествах данных.

## **Алгоритмы бандитов UCB и Successive Rejects**

В этом чанке наиболее важной концепцией является использование алгоритмов оптимизации, таких как UCB (Upper Confidence Bound) и Successive Rejects, для эффективного выбора лучших предложений в задачах обработки естественного языка (NLP). Эти алгоритмы помогают определить, какие варианты предложений (prompts) ведут к наилучшим результатам при минимальном количестве оценок (или "пулов").

### 1. Алгоритм UCB

UCB — это метод, который помогает быстро оценить производительность предложений, выбирая те, которые обещают наилучший результат на основе предыдущей информации. Основная идея алгоритма заключается в том, чтобы выбирать предложение, максимизируя его оценку производительности и добавляя некоторую "уверенность" в его выбор. Это можно представить формулой:

$$ 
\text{UCB}(p_i) = Q_t(p_i) + c \sqrt{\frac{\log t}{N_t(p_i)}} 
$$

где:
- $Q_t(p_i)$ — это оценка производительности предложения $p_i$ на момент времени $t$,
- $c$ — параметр исследования,
- $N_t(p_i)$ — общее число запросов к предложению $p_i$ до момента времени $t$,
- $t$ — текущий временной шаг.

Таким образом, алгоритм UCB комбинирует фактические результаты с некоторой степенью неопределенности, что позволяет избежать чрезмерной эксплуатации менееобещающих вариантов.

### 2. Алгоритм Successive Rejects

Successive Rejects — это другой подход, который последовательно отсекает менее перспективные предложения. Алгоритм проходит через $n-1$ фаз, в каждой из которых производится оценка предложений на случайных данных. В каждой фазе удаляется предложение с наименьшей оценкой. Это позволяет сосредоточиться на наиболее перспективных вариантах.

- В первой строке алгоритма $S_0$ задается как множество всех предложений, и затем происходит итерация по фазам:
  - Каждый кандидат оценивается на определенном подмножестве данных.
  - Удаляется предложение с наименьшим результатом. 

### Пример кода для UCB

```python
import numpy as np

# Пример реализации UCB
class UCB:
    def __init__(self, n_prompts):
        self.n_prompts = n_prompts                  # Общее количество предложений
        self.Nt = np.zeros(n_prompts)               # Общее число запросов к каждому предложению
        self.Qt = np.zeros(n_prompts)               # Оценка производительности каждого предложения

    def select(self, t, c):
        # Вычисление UCB для каждого предложения
        return np.argmax(self.Qt + c * np.sqrt(np.log(t) / (self.Nt + 1e-5)))

    def update(self, prompt, reward):
        self.Nt[prompt] += 1                       # Увеличиваем счетчик запросов для данного предложения
        self.Qt[prompt] += (reward - self.Qt[prompt]) / self.Nt[prompt]  # Обновляем оценку производительности
```

- В этом коде создается класс `UCB`, который хранит количество запросов и оценку для каждого предложения.
- Метод `select` вычисляет UCB для всех предложений и возвращает индекс с максимальным значением.
- Метод `update` обновляет оценку производительности на основе полученного вознаграждения от конкретного предложения.

В результате предлагаемые алгоритмы позволяют исследовать пространство возможных предложений и выбирать наиболее эффективные без необходимости массового тестирования всех вариантов, что сохраняет ресурсы и время.
### **Чанк 3:**

**Предыдущий контекст:** В предыдущем чанке обсуждались различные задачи классификации в области обработки естественного языка (NLP), такие как обнаружение атак на настройки языковых моделей и распознавание ненавистной речи или фейковых новостей. Теперь авторы статьи перейдут к описанию своего метода и его сравнении с другими существующими методами.

## **Оптимизация языка подсказок в задачах NLP**

В данном чанке основное внимание уделяется предложенной методологии для оптимизации языковых подсказок (prompts) в задачах обработки естественного языка (NLP). Метод основан на использовании алгоритма под названием ProTeGi и его сравнении с несколькими существующими методами.

### Основная концепция

Оптимизация языка подсказок представляет собой процесс улучшения текста подсказки, чтобы обеспечить более точные и релевантные ответы от моделей языкового понимания, таких как GPT-3.5. Подсказка может сильно повлиять на результаты, получаемые от модели, и через данный процесс можно направлять модель к более точным выводам при анализе текста.

Применяемая метрика при оптимизации - это $F1$-score, который представляет собой гармоническое среднее между точностью (precision) и полнотой (recall):

$$ F1 = 2 \cdot \frac{precision \cdot recall}{precision + recall} $$

где:
- **Точность (precision)**: это доля правильно предсказанных положительных примеров от общего числа предсказанных положительных примеров.
- **Полнота (recall)**: это доля правильно предсказанных положительных примеров от общего числа фактических положительных примеров.

Эта метрика очень полезна в задачах классового анализа, поскольку учитывает как ошибки первого рода (ложные срабатывания), так и второго рода (ложные отказы). 

### Пример кода

Давайте рассмотрим пример кода на Python, который использует библиотеку `sklearn` для расчета $F1$-score:

```python
from sklearn.metrics import f1_score

# Список предсказанных значений
y_pred = [1, 0, 1, 1, 0, 1]

# Список истинных значений
y_true = [1, 0, 0, 1, 0, 1]

# Вычисление F1-Score
f1 = f1_score(y_true, y_pred)

# Вывод результата
print(f'F1 Score: {f1:.2f}')  # Отображение F1-Score с двумя знаками после запятой
```

### Объяснение кода:

1. **Импорт библиотеки**: `from sklearn.metrics import f1_score` - здесь мы импортируем функцию `f1_score` из библиотеки `sklearn`, которая используется для вычисления значения F1-метрики.
2. **Создание списков предсказанных и истинных значений**: `y_pred` и `y_true` содержат предсказанные моделью и реальные значения соответственно.
3. **Вычисление F1-Score**: `f1 = f1_score(y_true, y_pred)` - вызывает функцию, которая считает $F1$-score на основе переданных истинных и предсказанных значений.
4. **Вывод результата**: `print(f'F1 Score: {f1:.2f}')` - выводит рассчитанное значение F1-метрики с двумя знаками после запятой.

Этот код хорошо иллюстрирует, как можно использовать метрики для оценки качества работы моделей в NLP задачах, что соответствует обсуждаемой концепции оптимизации подсказок в статье.
### Чанк 3:

**Предыдущий контекст:** Во втором чанке говорилось о сравнительном анализе различных алгоритмов, включая ProTeGi, и их производительности на различных наборах данных. Это привело к выводам о том, что ProTeGi способен значительно улучшить результаты по сравнению с другими методами.

## **Экспериментальные результаты**

Данный чанк фокусируется на представлении результатов экспериментов, которые показывают, что алгоритм ProTeGi превосходит другие современные алгоритмы на всех четырех рассматриваемых наборах данных. Он подчеркивает, что ProTeGi показал среднее улучшение на 3.9% и 8.2% по сравнению с базовыми алгоритмами MC и RL соответственно, и на 15.3% и 15.2% по сравнению с исходным.prompt $p_0$ и AutoGPT.

Важная часть заключается в том, что производительность алгоритмов снижается, когда количество оценок (или "evaluation") на кандидата понижается. Это подчеркивает, что для надежной работы алгоритмы требуют достаточного количества оценок для снижения вариативности результата.

Некоторые алгоритмы, такие как MC (монте-карло), могут стабильно улучшать производительность запросов, однако RL (обучение с подкреплением) и AutoPrompt показывают нестабильные результаты. Например, в некоторых случаях использование AutoGPT на протяжении шести раундов фактически снижало производительность исходного промпта.

Значение заключается в том, что различные техники оптимизации могут быть более подходящими для разных задач обработки естественного языка, и что подходы, подобные ProTeGi, могут быть необходимы для достижения оптимальных результатов.

### Математическая формализация

В этой части текста рассматриваются параметры, связанные с выбором кандидатов в алгоритмах. Например, для опции "bandit algorithms", используется параметр бюджетов $B$, который рассчитывается как:

$$B = T^* \cdot \frac{|D|}{n}$$ 

где $T^*$ - общее количество запросов, $|D|$ - размер выборки, $n$ - количество алгоритмов, участвующих в сравнении. Это указывает на то, как бюджет распределяется среди разных выборок для обеспечения равных условий.

### Пример и объяснение кода

Приведем пример кода, который иллюстрирует, как можно реализовать механизм выбора "лучшого ручья" (best arm) из набора алгоритмов. Этот код демонстрирует конкретную реализацию подхода UCB (Upper Confidence Bound).

```python
import numpy as np

class Bandit:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.counts = np.zeros(n_arms)  # Счетчик количеств выборов для каждой руки
        self.values = np.zeros(n_arms)  # Оцененные значения каждой руки

    def select_arm(self):
        total_counts = sum(self.counts)
        if total_counts == 0:
            return np.random.choice(self.n_arms)  # Первый выбор, случайная рука
        ucb_values = self.values + np.sqrt(2 * np.log(total_counts) / self.counts)  # Вычисляем UCB
        return np.argmax(ucb_values)  # Возвращаем руку с максимальным UCB

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] += 1  # Увеличиваем счетчик для выбранной руки
        n = self.counts[chosen_arm]
        value = self.values[chosen_arm]
        # Обновляем оцененное значение руки
        self.values[chosen_arm] = ((n - 1) / n) * value + (1 / n) * reward  # Обновление по формуле средней

# Пример использования Bandit
n_arms = 3  # Количество выборов
bandit = Bandit(n_arms)

for _ in range(100):  # 100 итераций
    arm = bandit.select_arm()  # Выбор руки
    reward = np.random.rand()  # Симулируем вознаграждение
    bandit.update(arm, reward)  # Обновляем статистику по руке
```

- **Импортируем библиотеку numpy** для работы с массивами.
- **Определяем класс Bandit**, который представляет собой механизм выбора "лучшем ручья".
- **Метод `select_arm`** выбирает руку, у которой максимальное значение UCB.
- **Метод `update`** обновляет информацию о выбранной руке на основе полученного вознаграждения.

Этот код демонстрирует, как алгоритм может адаптироваться и оптимизироваться в зависимости от полученных данных, аналогично описанному в чанкe о ProTeGi и его сравнении с другими подходами.
### **Чанк 1:**

## **Сравнение الگорифмов: UCB и Successive Rejects**

В данном чанке рассматриваются различные алгоритмы для решения задачи мульти-рукавного бандита, включая UCB (Upper Confidence Bound) и Successive Rejects. Это основные методы, используемые для эффективного распределения ресурсов между различными опциями.

Алгоритмы мульти-рукавного бандита помогают принимать решения в условиях неопределенности, когда необходимо оптимально выбирать из большого числа вариантов. Например, если у вас есть несколько рекламных объявлений, алгоритм может помочь определить, какое из них приносит больше всего кликов, при этом рассматривая как текущие результаты, так и потенциальные возможности.

### **Ключевая концепция:** Алгоритмы мульти-рукавного бандита

Алгоритмы мульти-рукавного бандита — это методы, которые позволяют балансировать между исследованием (exploration) и эксплуатацией (exploitation) в задачах принятия решений. Исследование подразумевает тестирование новых вариантов, в то время как эксплуатация ориентирована на использование уже известных "лучших" вариантов. 

Из результата исследования видно, что один из алгоритмов, UCB, показывает более высокую эффективность в сравнении с традиционными методами, такими как Successive Rejects. Это можно объяснить тем, что UCB лучше справляется с балансировкой между исследованием и эксплуатацией, используя параметр $c$, чтобы адаптироваться к ситуации.

### **Математическая формализация:** 

Формула для алгоритма UCB выглядит следующим образом:
$$
UCB_i = \bar{X}_i + c \sqrt{\frac{2 \ln n}{n_i}}
$$
где:
- $UCB_i$ — оценка для i-го "рукава" (опции);
- $\bar{X}_i$ — средняя награда, полученная от i-го "рукава";
- $n$ — общее количество испытаний;
- $n_i$ — количество испытаний i-го "рукава";
- $c$ — параметр, определяющий степень исследования.

Каждая часть формулы имеет свое значение:
- Первый элемент $(\bar{X}_i)$ представляет собой среднюю эффективность данной опции.
- Второй элемент $(c \sqrt{\frac{2 \ln n}{n_i}})$ стимулирует исследование, увеличиваясь, когда $n_i$ меньше, позволяя алгоритму попробовать новые опции.

### **Пример кода:**

Вот пример реализация простейшего алгоритма UCB на Python:

```python
import math

class Bandit:
    def __init__(self, num_options):
        self.num_options = num_options  # Общее количество "рукавов"
        self.counts = [0] * num_options  # Количество испытаний для каждого "рукава"
        self.values = [0.0] * num_options  # Средняя награда для каждого "рукава"

    def update(self, option, reward):
        # Обновление значений после испытания опции
        self.counts[option] += 1  # Увеличиваем количество испытаний для выбранного "рукава"
        # Обновляем среднее значение награды
        n = self.counts[option]
        value = self.values[option]
        self.values[option] = value + (reward - value) / n

    def select(self):
        # Выбор "рукава" по формуле UCB
        total_counts = sum(self.counts)
        ucb_values = [
            value + math.sqrt(2 * math.log(total_counts) / (count + 1e-5)) if count > 0 else float('inf')
            for value, count in zip(self.values, self.counts)
        ]
        return ucb_values.index(max(ucb_values))  # Возвращаем индекс наибольшего UCB

# Пример использования
bandit = Bandit(num_options=5)

# Симуляция обновления и выбора
for _ in range(100):
    option = bandit.select()  # Выбор "рукава"
    reward = ...  # Получение награды от выбранной опции
    bandit.update(option, reward)  # Обновление значений
```

Каждая строка кода в этом примере выполняет специфическую задачу:
- Мы создаем класс `Bandit`, который будет управлять различными опциями.
- Строки кода в методе `update` обновляют среднюю награду для выбранной опции.
- Метод `select` использует формулу UCB для выбора наилучшей опции на основе текущих средних значений и количества испытаний.

Таким образом, алгоритмы мульти-рукавного бандита, такие как UCB, предлагают мощные инструменты для оптимизации выборов в условиях неопределенности.
### **Чанк 4:**

**Предыдущий контекст:** В предыдущем чанке обсуждались различные методы улучшения подсказок для языковых моделей (LLM), включая дифференцируемую настройку мягких подсказок и обучение вспомогательных моделей. В этом чанкe рассматривается связь между предложенным методом ProTeGi и существующими подходами в области оптимизации подсказок.

## **Оптимизация подсказок для языковых моделей (ProTeGi)**

В данном разделе представлена новая концепция под названием ProTeGi (Prompt Optimization with Textual Gradients), которая фокусируется на автоматической оптимизации подсказок для LLM. Основная идея состоит в том, что традиционные методы оптимизации, такие как дифференцируемая настройка и использование вспомогательных моделей, имеют свои ограничения, поскольку часто требуют доступа к внутренним переменным модели или зависят от числовых функций награды. ProTeGi направлен на устранение этих проблем.

### Ключевые аспекты ProTeGi:
- ProTeGi использует текстовые градиенты, чтобы преодолеть барьер дискретной оптимизации.
- Метод сочетает в себе шаги градиентного спуска (процесс нахождения локального минимума функции) в текстовом диалоге.
- ProTeGi применяет поиск по «лучевым» (beam search) пространствам подсказок с использованием эффективного выбора бандита (bandit selection), что позволяет улучшать подсказки.

#### Математическая формализация:
Чтобы объяснить, как работает метод, рассмотрим формулу для градиентного спуска:

$$w_{t+1} = w_t - \alpha \nabla J(w_t)$$

где:
- $w_t$ – текущее значение параметров (например, весов модели),
- $\alpha$ – скорость обучения (learning rate),
- $\nabla J(w_t)$ – градиент функции потерь по отношению к параметрам.

Эта формула иллюстрирует, как параметры модели обновляются на каждой итерации $t$ на основе градиента функции потерь. В контексте ProTeGi, мы можем рассмотреть параметры как текстовые подсказки, которые обновляются на основе текстовых градиентов.

### Пример кода:
Для иллюстрации процесса оптимизации подсказок, представим простую реализацию градиентного спуска для текстовых подсказок. Предположим, что мы хотим оптимизировать подсказку для получения корректного ответа от LLM. 

```python
import numpy as np

# Изначальная подсказка
prompt = "How does climate change affect weather patterns?"
# Функция для получения ответа от LLM
def get_response(prompt):
    # Симуляция LLM, возвращающей случайный ответ
    return np.random.choice(["It's getting worse.", "Not much is changing.", "Many regions are affected."])

# Градиентная функция, показывающая насколько ответ хороший
def gradient_response(response):
    return len(response)  # Простой пример: длина ответа - наш «градиент»

# Обновление подсказки на основе градиента
learning_rate = 0.1
for i in range(10):  # 10 итераций
    response = get_response(prompt)
    grad = gradient_response(response)
    prompt = prompt + " " + "improve" * int(grad * learning_rate)  # Модификация подсказки
    print(f"Iteration {i+1}: New Prompt: {prompt}")
```

#### Объяснение кода:
- Изначально задается подсказка `prompt`.
- Функция `get_response` симулирует работу LLM, возвращая случайный ответ.
- Функция `gradient_response` определяет «градиент» как длину ответа. Это очень упрощенная модель.
- Цикл обновляет подсказку на основе полученного градиента в течение 10 итераций, добавляя слова «improve» в зависимости от градиента.
- В конце каждой итерации выводится новая подсказка.

Таким образом, ProTeGi предлагает гибкий и независимый от модели метод оптимизации, который можно применять для усовершенствования подсказок в широком круге задач без необходимости сложного обучения или тонкой настройки гиперпараметров.
### **Чанк 5:**

**Предыдущий контекст:** В предыдущем чанке обсуждалось использование технологии ProTeGi для оптимизации текстовых запросов, а также выявленные ограничения, такие как зависимость от API и необходимость в больших вычислительных ресурсах для выполнения задач.

## **Будущие направления работы и ограничения ProTeGi**

В этом чанке ключевой концепцией являются **будущие направления работы** и **ограничения** использования ProTeGi, что касается как его применения, так и области тестирования. 

1. **Будущие направления работы**: Всегда есть место для улучшений. Авторы подчеркивают, что есть необходимость в обобщении текущих техник, чтобы они могли быть применены к большему количеству задач и использовать новые метрики. Это предполагает возможность интеграции различных переменных, таких как размеры шага в обучения, что может повысить гибкость и эффективность обучения.

2. **Ограничения**: 
   - **Проблемы с производительностью**: Одним из ключевых ограничений является зависимость от API. Частые запросы к API, особенно в больших количествах, могут приводить к увеличению времени выполнения задач, иногда превышающему час. Это делает ProTeGi менее подходящим для срочных приложений.
   - **Ограниченное тестирование**: ProTeGi был протестирован только на четырех классификационных задачах. Это создает возможные ограничения в понимании универсальности и эффективности метода в других, более сложных доменах.

Авторы подчеркивают, что дальнейшее тестирование и доработка ProTeGi необходимы для работы в сложных моделях, которые могут выходить за рамки тех задач, на которых он был изначально протестирован.

### Примечания:

- Все выводы о будущем ProTeGi подразумевают, что исследователи будут стремиться расширять и улучшать его использование, чтобы преодолеть ограничения и применить их в новых задачах.
  
- Изучение возможных решений для проблемы с производительностью может включать внедрение более эффективных алгоритмов или архитектур, которые могли бы оптимизировать количество запросов к API. 

В заключение, понимание ограничений и возможностей улучшения ProTeGi является критически важным для будущих исследований в данном направлении.
### **Чанк 1:**

## **Градиентные знакомства (Gradient Prompts)**

В этом чанке рассматривается использование заданий (prompts) для генерации градиентов и улучшения нулево-выборочных классификаторов. Основная концепция заключается в том, как формулировка задания может влиять на качество и точность модели при перечислении ошибок и их причинах.

Процесс создания задания для генерации градиентов выглядит следующим образом. Задание начинается со стандартной строки, которая включает переменные, такие как текущий текст задания и строки ошибок модели. Это подходит для создания контекста, в котором модель может указать, что пошло не так, и предложить возможные улучшения.

#### Этапы объяснения и применение концепции:
1. **Основная структура задания:** Запрос начинается с "Я пытаюсь написать нулево-выборочный классификатор". Затем указывается текущее задание и строки ошибок, которые модель допустила.
2. **Градиенты:** Модель генерирует ответы, где указывается, почему задание не сработало, причем каждая причина обрамляется специальными маркерами (`<START>` и `<END>`).
3. **Динамическая подстановка:** Переменные, такие как текущее задание и строки ошибок, подставляются в задании в реальном времени, что делает модель более адаптивной.

#### Математическая формализация:
Нет сложных математических формул в этом чанке, однако важно понимать, что данные строки представляют тексты в качестве переменных, которые могут быть включены динамически в запросы.

#### Пример кода:
```python
def generate_gradient_prompt(prompt, error_strings, num_feedbacks):
    """
    Генерирует текст задания для получения градиентов.
    
    :param prompt: Текущее задание
    :param error_strings: Ошибки, которые текущая модель сгенерировала
    :param num_feedbacks: Количество запрашиваемых отзывов
    :return: Созданное задание для генерации градиента
    """
    
    # Формируем базовый шаблон запроса
    base_prompt = (
        "I'm trying to write a zero-shot classifier prompt.\n"
        f"My current prompt is:\n\"{prompt}\"\n"
        f"But this prompt gets the following examples wrong:\n{error_strings}\n"
        f"give {num_feedbacks} reasons why the prompt could have gotten these examples wrong.\n"
        "Wrap each reason with <START> and <END>"
    )
    
    return base_prompt

# Пример использования функции
prompt = "classify news articles"
error_strings = "The model misclassified the following headlines."
feedback_count = 3

# Генерируем задание
generated_prompt = generate_gradient_prompt(prompt, error_strings, feedback_count)
print(generated_prompt)
```
В этом коде:
- Функция `generate_gradient_prompt` принимает текущее задание, строки ошибок и количество запрашиваемых отзывов.
- Она формирует строку на основе шаблона с подставленными значениями.
- Возвращаемый результат - это текст задания, который готов к использованию в модели.
### **Чанк 1:**

## **Ключевая концепция: Инициализация промтов для LLM**

В данном чанкe обсуждаются начальные промты (или запросы), которые были составлены профессиональными инженерами машинного обучения для моделирования поведения крупных языковых моделей (LLM). Эти промты используются для решения различных задач, таких как обнаружение атак на систему (jailbreak), определение ненавистной речи (hate speech), определение лжи и сарказма.

### Объяснение концепции

Инициализация промтов играет ключевую роль в том, как языковая модель взаимодействует с вводимыми данными. Промт — это приказ или вопрос, который задается модели, чтобы получить нужный ответ. Например, в случае задачи "jailbreak" от модели ожидается ответ "Да" или "Нет" в зависимости от того, является ли сообщение попыткой взлома системы. 

Промты определяют, как модель интерпретирует запрос и формулирует ответ. Таким образом, правильная и четкая формулировка промтов критически важна для достижения точных результатов.

### Примеры промтов

1. **Jailbreak**
   - **Задача:** Обнаружить, является ли сообщение атакой на систему.
   - **Формат вывода:** Ответ "Да" или "Нет".

2. **Ethos**
   - **Задача:** Определить, является ли данный текст ненавистной речью.
   - **Формат вывода:** Ответ "Да" или "Нет".

Эти примеры показывают, как можно структурировать запросы к модели для выполнения конкретной задачи.

### Математическая формализация

На данном этапе нет явных математических формул, поскольку основной акцент делается на структуре промтов и их задачах.

### Примеры и объяснение кода

На текущем этапе текст не содержит кода, но представим, как мог бы выглядеть пример кода, который реализует базовую логику для определения, является ли текст ненавистной речью. Предположим, у нас есть базовая функция для этого:

```python
def is_hate_speech(text):
    # Предполагаем, что у нас есть список ненавистных слов
    hate_speech_words = ['hate', 'violence', 'kill']  
    # Приводим текст к нижнему регистру для упрощения сравнения
    text_lower = text.lower()

    # Проверяем, содержится ли какое-либо слово из списка в тексте
    for word in hate_speech_words:
        if word in text_lower:
            return "Yes"  # Возвращаем "Да", если найдено ненавистное слово
    return "No"  # Возвращаем "Нет", если ненавистных слов не найдено
```

- **Объяснение кода:**
  - `hate_speech_words`: Список слов, которые мы будем использовать для определения ненавистной речи.
  - `text_lower`: Преобразует текст в нижний регистр для упрощения сравнения.
  - `for word in hate_speech_words`: Цикл проходит по каждому слову в списке.
  - `if word in text_lower`: Проверяет, содержится ли слово в тексте; если да, функция возвращает "Да".
  - Если никакие ненавистные слова не найдены, функция возвращает "Нет".

Этот код иллюстрирует, как можно применять логику, аналогичную описанным задачам в промтах, для автоматического анализа текста.
### **Чанк 1:**

## **Ключевая концепция: Искажение фактов (Ложь)**

В первом чанке обсуждается процесс оценки истинности заявлений на основе контекста и других факторов, включая потенциальные предвзятости оратора. Например, в ситуации, когда сенатор из Техаса делает заявление о том, что "малые предприятия закрываются рекордными темпами", важно учитывать, что его партийная принадлежность и позиция могут влиять на правдивость его слов.

Подробнее о концепции: Искажение фактов происходит, когда информация не соответствует реальности, часто из-за предвзятости или намеренного искажения. В контексте заявлений политиков важно учитывать, что такие лица могут представлять интересы своей партии или ожидания избирателей.

Когда мы говорим о том, чтобы определить, является ли утверждение ложным (Да) или правдивым (Нет), мы должны анализировать контекст, в котором оно сделано, и любые доступные факты, которые могут проверить это утверждение. Например, можно использовать данные о количестве малых предприятий, закртых за определенный период, чтобы подтвердить или опровергнуть заявление сенатора.

### **Чанк 2:**

**Предыдущий контекст:** В предыдущем чанке обсуждалось искажение фактов и важность анализа контекста, в котором делаются заявления, при определении их истинности.

## **Обнаружение сарказма**

В этом чанке рассматривается, как классифицировать сообщения на предмет сарказма и их принадлежности к попыткам обойти защитные механизмы системы ИИ. Сарказм может быть сложным для распознавания, так как он часто зависит от контекста и интонации, которые сложно передать через текст.

Помимо этого, здесь обсуждается, что некоторые сообщения могут представлять собой попытки провести джейлбрейк-атаки (jailbreak attack) — это попытки пользователей обойти защитные механизмы системы. Например, в приведенном сообщении о "блуждающих собаках" использование метафор может указывать на сарказм или насмешку, что делает необходимым анализировать тон и выбор слов.

Некоторые подходы к обнаружению сарказма могут включать машинное обучение, где модели обучаются на различных примерах и контексте, чтобы различать прямые и ироничные выражения. Основные признаки, которые могут помочь в этом, включают эмоциональную окраску слова и структуру предложения.

### Примечание

На этом этапе нет использования конкретных математических формул или кода, так как обсуждаются скорее концептуальные вопросы без строгих количественных показателей. Тем не менее, в дальнейшем, если будут обсуждаться алгоритмы или методы, можно будет приводить примеры кода для реализации оценки истинности или распознавания сарказма.
